In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [ ]:
from datasets import load_dataset, load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd

dataset = load_dataset("cnn_dailymail", "3.0.0")

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print("The Structure of the dataset:")
print(dataset)

The Structure of the dataset:
DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [ ]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize summaries
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def summarize_text(text, model, tokenizer, max_length=150, min_length=40, num_beams=4):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=num_beams, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
articles = dataset['train']['article'][:5]
highlights = dataset['train']['highlights'][:5]

In [ ]:
# Create lists to store the results
data = {
    'Original Article': [],
    'Original Highlights': [],
    'Generated Summary': []
}

In [ ]:
# Iterate over articles and highlights
for article, highlight in zip(articles, highlights):
    data['Original Article'].append(article)
    data['Original Highlights'].append(highlight)
    generated_summary = summarize_text(article, model, tokenizer)
    data['Generated Summary'].append(generated_summary)

In [ ]:
# Create a DataFrame from the data
df_results = pd.DataFrame(data)

In [34]:
df_results

,Original Article,Original Highlights,Generated Summary
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,the young actor says he has no plans to fritte...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,inmates with most severe mental illnesses are ...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...","survivor: ""i probably had a 30-, 35-foot free ..."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",the polyps were removed and sent to the nation...
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...","new: ""your admitted conduct was not only illeg..."


In [ ]:
# Function to summarize text for evaluation
def summarize_text_eval(text, model, tokenizer, max_length=300, min_length=40):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [28]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00


In [31]:
import evaluate
def evaluate_model(model, tokenizer, dataset, num_samples=50):
    articles = dataset['validation']['article'][:num_samples]
    references = dataset['validation']['highlights'][:num_samples]

    summaries = [summarize_text_eval(article, model, tokenizer) for article in articles]

    # Load ROUGE metric
    rouge = evaluate.load('rouge')

    # Compute ROUGE scores
    results = rouge.compute(predictions=summaries, references=references)
    return results

In [33]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=fc2bfd3dde028242609176380917b989ad0f5375340715f7d683681fe70e2b1a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [37]:
import pandas as pd
import evaluate
results = evaluate_model(model, tokenizer, dataset, num_samples=50)

In [43]:
def format_results(results):
    import pandas as pd

    # Extract scores from nested structure if necessary
    rouge1_score = results['rouge1']['mid']['fmeasure'] if isinstance(results['rouge1'], dict) else results['rouge1']
    rouge2_score = results['rouge2']['mid']['fmeasure'] if isinstance(results['rouge2'], dict) else results['rouge2']
    rougeL_score = results['rougeL']['mid']['fmeasure'] if isinstance(results['rougeL'], dict) else results['rougeL']

    # Create a DataFrame with the appropriate structure
    df_results = pd.DataFrame({
        'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
        'Score': [
            rouge1_score,
            rouge2_score,
            rougeL_score
        ]
    })

    return df_results



In [44]:
# Format and print the results
df_eval_results = format_results(results)
print(df_eval_results)

    Metric     Score
0  ROUGE-1  0.306527
1  ROUGE-2  0.142991
2  ROUGE-L  0.242000


In [45]:
df_eval_results

,Metric,Score
0,ROUGE-1,0.306527
1,ROUGE-2,0.142991
2,ROUGE-L,0.242000
